In [76]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [77]:
from datetime import timedelta

In [78]:
original_flight_schedule_path = "/kaggle/input/flight-schedule-2-nov/Flight Schedule 2 Nov.xlsx"

In [79]:
original_flight_schedule = pd.read_excel(original_flight_schedule_path)

In [80]:
original_flight_schedule.head()

,TABBID,FLTNO,TAIL,BODY,DEP,DEP_CITY,ARR,ARR_CITY,RTDIST,RTAIST
0,11614,ZZ0727,VTAIA,NB319,BLR,Bangalore,GAU,Gauhati,02-Nov-2020 13:00,02-Nov-2020 16:00
1,11615,ZZ0728,VTAIA,NB319,GAU,Gauhati,BLR,Bangalore,02-Nov-2020 17:00,02-Nov-2020 20:20
2,11348,ZZ0701,VTAIB,NB319,CCU,Kolkata,DEL,New Delhi,02-Nov-2020 17:10,02-Nov-2020 19:25
3,11349,ZZ0702,VTAIB,NB319,DEL,New Delhi,CCU,Kolkata,02-Nov-2020 21:15,02-Nov-2020 23:30
4,10918,ZZ0640,VTAIF,NB319,BLR,Bangalore,BOM,Mumbai,02-Nov-2020 07:30,02-Nov-2020 09:20


In [81]:
optimal_flight_schedule_path_1 = "/kaggle/input/optimal-nov-2/Output_VTAIA_BLR_1280.csv"

In [82]:
optimal_flight_schedule_1 = pd.read_csv(optimal_flight_schedule_path_1)

In [83]:
optimal_flight_schedule_1.head()

,TAIL,TABBID,FLT,DEP,ARR,RTDIST,RTAIST
0,VTICD,3486,ZZ0560,DEL,HYD,7:15,9:10
1,VTICD,3487,ZZ0543,HYD,DEL,11:10,13:45
2,VTICD,3488,ZZ0415,DEL,PAT,17:0,18:40
3,VTICD,3489,ZZ0416,PAT,DEL,19:30,21:10
4,VTICF,3742,ZZ0776,BLR,CCU,8:10,10:40


In [84]:
# Function to discretize the time
def discretize_time(time):
    return time.replace(minute=0 if time.minute < 30 else 30, second=0, microsecond=0)

### Create unique flight identifier in the original schedule

In [85]:
# Applying the preprocessing steps to the original schedule
original_flight_schedule['Scheduled Departure Time'] = pd.to_datetime(original_flight_schedule['RTDIST'], format='%d-%b-%Y %H:%M', errors='coerce')
original_flight_schedule['Scheduled Departure Time'] = original_flight_schedule['Scheduled Departure Time'].apply(lambda t: t + timedelta(hours=3) if t.hour < 3 else t)
original_flight_schedule['Discretized Departure Time'] = original_flight_schedule['Scheduled Departure Time'].apply(discretize_time)

# Creating a unique identifier for each flight in the original schedule
original_flight_schedule['Flight Identifier'] = (
    original_flight_schedule['DEP'] + "-" + original_flight_schedule['ARR'] + "-" + original_flight_schedule['Discretized Departure Time'].astype(str) + "-" + original_flight_schedule['TAIL'].astype(str)
)

In [86]:
original_flight_schedule.head(2)

,TABBID,FLTNO,TAIL,BODY,DEP,DEP_CITY,ARR,ARR_CITY,RTDIST,RTAIST,Scheduled Departure Time,Discretized Departure Time,Flight Identifier
0,11614,ZZ0727,VTAIA,NB319,BLR,Bangalore,GAU,Gauhati,02-Nov-2020 13:00,02-Nov-2020 16:00,2020-11-02 13:00:00,2020-11-02 13:00:00,BLR-GAU-2020-11-02 13:00:00-VTAIA
1,11615,ZZ0728,VTAIA,NB319,GAU,Gauhati,BLR,Bangalore,02-Nov-2020 17:00,02-Nov-2020 20:20,2020-11-02 17:00:00,2020-11-02 17:00:00,GAU-BLR-2020-11-02 17:00:00-VTAIA


In [87]:
original_flight_schedule['Flight Identifier'].nunique()

136

So, there are 136 unique flight identifier in the original schedule

### Create unique flight identifier in the optimal schedule

In [88]:
# Preprocessing: Convert times into a uniform format
optimal_flight_schedule_1['Actual Departure Time'] = pd.to_datetime(optimal_flight_schedule_1['RTDIST'], format='%H:%M', errors='coerce').apply(lambda t: t.replace(year=2020, month=11, day=2))
optimal_flight_schedule_1['Actual Departure Time'] = optimal_flight_schedule_1['Actual Departure Time'].apply(lambda t: t + timedelta(hours=3) if t.hour < 3 else t)
optimal_flight_schedule_1['Discretized Departure Time'] = optimal_flight_schedule_1['Actual Departure Time'].apply(discretize_time)

# Creating a unique identifier for each flight combining origin, destination, discretized time and tail
optimal_flight_schedule_1['Flight Identifier'] = (
    optimal_flight_schedule_1['DEP'] + "-" + optimal_flight_schedule_1['ARR'] + "-" + optimal_flight_schedule_1['Discretized Departure Time'].astype(str) + "-" + optimal_flight_schedule_1['TAIL'].astype(str)
)

In [89]:
optimal_flight_schedule_1['Flight Identifier'].nunique()

134

In [90]:
optimal_flight_schedule_1.head(2)

,TAIL,TABBID,FLT,DEP,ARR,RTDIST,RTAIST,Actual Departure Time,Discretized Departure Time,Flight Identifier
0,VTICD,3486,ZZ0560,DEL,HYD,7:15,9:10,2020-11-02 07:15:00,2020-11-02 07:00:00,DEL-HYD-2020-11-02 07:00:00-VTICD
1,VTICD,3487,ZZ0543,HYD,DEL,11:10,13:45,2020-11-02 11:10:00,2020-11-02 11:00:00,HYD-DEL-2020-11-02 11:00:00-VTICD


In [91]:
# Extract the unique flight identifiers from the optimal schedule
unique_flight_identifiers_optimal = optimal_flight_schedule_1['Flight Identifier'].unique()
unique_flight_identifiers_optimal[:3]

array(['DEL-HYD-2020-11-02 07:00:00-VTICD',
       'HYD-DEL-2020-11-02 11:00:00-VTICD',
       'DEL-PAT-2020-11-02 17:00:00-VTICD'], dtype=object)

#### Creating the y1 label

In [92]:
# Create a binary vector for the original schedule
y1 = original_flight_schedule['Flight Identifier'].apply(lambda x: 0 if x in unique_flight_identifiers_optimal else 1)

# Extracting the binary vector
original_schedule_feature_vector = y1.to_numpy()

# Displaying the shape of the binary vector and the first few elements as an example
original_schedule_feature_vector.shape, original_schedule_feature_vector[:5]

((136,), array([1, 1, 0, 0, 0]))

In [93]:
original_schedule_feature_vector

array([1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [94]:
def helper_fun(optimal_schedule_path):
    
    optimal_schedule = pd.read_csv(optimal_schedule_path)
    
    # Preprocessing: Convert times into a uniform format
    optimal_schedule['Actual Departure Time'] = pd.to_datetime(optimal_schedule['RTDIST'], format='%H:%M', errors='coerce').apply(lambda t: t.replace(year=2020, month=11, day=2))
    optimal_schedule['Actual Departure Time'] = optimal_schedule['Actual Departure Time'].apply(lambda t: t + timedelta(hours=3) if t.hour < 3 else t)
    optimal_schedule['Discretized Departure Time'] = optimal_schedule['Actual Departure Time'].apply(discretize_time)
    
    # Creating a unique identifier for each flight combining origin, destination, discretized time and tail
    optimal_schedule['Flight Identifier'] = (
                optimal_schedule['DEP'] 
                + "-" 
                + optimal_schedule['ARR'] 
                + "-" 
                + optimal_schedule['Discretized Departure Time'].astype(str) 
                + "-" 
                + optimal_schedule['TAIL'].astype(str)
    )
    
    # Extract the unique flight identifiers from the optimal schedule
    unique_flight_identifiers_optimal = optimal_schedule['Flight Identifier'].unique()
    
    y = original_flight_schedule['Flight Identifier'].apply(lambda x: 1 if x in unique_flight_identifiers_optimal else 0)
    
    # Extracting the binary vector
    original_schedule_feature_vector = y.to_numpy()
    return y

In [95]:
optimal_flight_schedule_path_2 = '/kaggle/input/optimal-nov-2/Output_VTAIA_BLR_1280.csv'
print(list(helper_fun(optimal_flight_schedule_path_2)))

[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


#### Generating the y label for all the optimal solution

In [96]:
optimal_schedules_folder_path = '/kaggle/input/optimal-nov-2'
optimal_schedule_files = [file for file in os.listdir(optimal_schedules_folder_path) if file.endswith('.csv')]

In [97]:
Y = pd.DataFrame()

In [98]:
from datetime import datetime, timedelta

# Function to discretize times into 30-minute intervals
def discretize_time(time):
    return time.replace(minute=0 if time.minute < 30 else 30, second=0, microsecond=0)

# Load and preprocess the original schedule
original_schedule_path = '/kaggle/input/flight-schedule-2-nov/Flight Schedule 2 Nov.xlsx'
original_schedule = pd.read_excel(original_schedule_path)
original_schedule['Scheduled Departure Time'] = pd.to_datetime(original_schedule['RTDIST'], format='%d-%b-%Y %H:%M', errors='coerce')
original_schedule['Scheduled Departure Time'] = original_schedule['Scheduled Departure Time'].apply(lambda t: t + timedelta(hours=3) if t.hour < 3 else t)
original_schedule['Discretized Departure Time'] = original_schedule['Scheduled Departure Time'].apply(discretize_time)
original_schedule['Flight Identifier'] = (
    original_schedule['DEP'] + "-" + original_schedule['ARR'] + "-" + original_schedule['Discretized Departure Time'].astype(str) + "-" + original_schedule['TAIL'].astype(str)
)

# Path to the folder containing all the optimal schedule files
optimal_schedules_folder_path = '/kaggle/input/optimal-nov-2'

# Getting a list of all the CSV files in the folder
optimal_schedule_files = [file for file in os.listdir(optimal_schedules_folder_path) if file.endswith('.csv')]

# Initialize a list to store the binary columns for each optimal schedule
binary_columns = []

# Iterate over each file and perform the necessary operations
for file in optimal_schedule_files:
    # Load the optimal schedule
    optimal_schedule_path = os.path.join(optimal_schedules_folder_path, file)
    optimal_schedule = pd.read_csv(optimal_schedule_path)

    # Preprocess the optimal schedule
    optimal_schedule['Actual Departure Time'] = pd.to_datetime(optimal_schedule['RTDIST'], format='%H:%M', errors='coerce').apply(lambda t: t.replace(year=2020, month=11, day=2))
    optimal_schedule['Actual Departure Time'] = optimal_schedule['Actual Departure Time'].apply(lambda t: t + timedelta(hours=3) if t.hour < 3 else t)
    optimal_schedule['Discretized Departure Time'] = optimal_schedule['Actual Departure Time'].apply(discretize_time)
    optimal_schedule['Flight Identifier'] = (
        optimal_schedule['DEP'] + "-" + optimal_schedule['ARR'] + "-" + optimal_schedule['Discretized Departure Time'].astype(str) + "-" + optimal_schedule['TAIL'].astype(str)
    )

    # Extract the unique flight identifiers from the optimal schedule
    unique_flight_identifiers_optimal = set(optimal_schedule['Flight Identifier'].unique())

    # Create a binary vector for the original schedule for this file
    binary_column = original_schedule['Flight Identifier'].apply(lambda x: 0 if x in unique_flight_identifiers_optimal else 1)
    binary_columns.append(binary_column.rename(f'Exists in {file}'))

# Concatenate all binary columns at once
Y = pd.concat(binary_columns, axis=1)

Y.head()

,Exists in Output_VTICD_DEL_1105.csv,Exists in Output_VTXEQ_DEL_743.csv,Exists in Output_VTXER_AUH_1607.csv,Exists in Output_VTICN_HYD_995.csv,Exists in Output_VTXEI_DEL_945.csv,Exists in Output_VTICO_MCT_364.csv,Exists in Output_VTBBE_DEL_965.csv,Exists in Output_VTXEA_DEL_1434.csv,Exists in Output_VTXED_DEL_937.csv,Exists in Output_VTXEO_BOM_1508.csv,...,Exists in Output_VTXEF_DEL_1374.csv,Exists in Output_VTBBJ_HYD_681.csv,Exists in Output_VTAII_CCU_1412.csv,Exists in Output_VTXES_DEL_826.csv,Exists in Output_VTAIA_BLR_1618.csv,Exists in Output_VTBBM_BOM_1138.csv,Exists in Output_VTXEU_BOM_846.csv,Exists in Output_VTAIR_DEL_675.csv,Exists in Output_VTBBM_BOM_1349.csv,Exists in Output_VTBBQ_DEL_786.csv
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [99]:
Y.shape

(136, 210)

In [100]:
Y = Y.T

In [101]:
Y.head()

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
Exists in Output_VTICD_DEL_1105.csv,1,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Exists in Output_VTXEQ_DEL_743.csv,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Exists in Output_VTXER_AUH_1607.csv,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,1,1,1,0,0
Exists in Output_VTICN_HYD_995.csv,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Exists in Output_VTXEI_DEL_945.csv,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [102]:
Y.shape

(210, 136)

In [103]:
# Print one y1 to verify
print(list(Y.loc["Exists in Output_VTAIA_BLR_1280.csv"]))

[1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Creating the X feature vectors

##### One hot encoding the disrupted flight name

In [104]:
# Path to the folder containing all the optimal schedule files
optimal_schedules_folder_path = '/kaggle/input/optimal-nov-2'

# Get a list of all the CSV files in the folder
optimal_schedule_files = [file for file in os.listdir(optimal_schedules_folder_path) if file.endswith('.csv')]

# Extract aircraft names from the filenames and get unique names
aircraft_names = [filename.split('_')[1] for filename in optimal_schedule_files]
unique_aircraft_names = list(set(aircraft_names))

# Create a DataFrame with one row for each file and one column for each unique aircraft
# Initially, all values are set to 0
one_hot_encoded_df = pd.DataFrame(0, index=optimal_schedule_files, columns=unique_aircraft_names)

# Iterate over each file and set the value to 1 for the corresponding aircraft in that file
for file in optimal_schedule_files:
    aircraft_name = file.split('_')[1]
    one_hot_encoded_df.at[file, aircraft_name] = 1

# Display the first few rows of the one-hot encoded DataFrame
one_hot_encoded_df.head()

,VTICN,VTICQ,VTICF,VTBBJ,VTBBT,VTAIO,VTAIB,VTXEI,VTXER,VTXEQ,...,VTAIP,VTICM,VTXDX,VTXEL,VTBBD,VTXEA,VTXET,VTBBX,VTICO,VTXEF
Output_VTICD_DEL_1105.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Output_VTXEQ_DEL_743.csv,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Output_VTXER_AUH_1607.csv,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Output_VTICN_HYD_995.csv,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Output_VTXEI_DEL_945.csv,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
one_hot_encoded_df.shape

(210, 42)

##### One hot encoding the departing place

In [106]:
# Extract departing place names from the filenames
departing_places = [filename.split('_')[2] for filename in optimal_schedule_files]
unique_departing_places = list(set(departing_places))

# Create a DataFrame for one-hot encoding the departing places
one_hot_encoded_departing_places = pd.DataFrame(0, index=optimal_schedule_files, columns=unique_departing_places)

# Iterate over each file and set the value to 1 for the corresponding departing place in that file
for file in optimal_schedule_files:
    departing_place = file.split('_')[2]
    one_hot_encoded_departing_places.at[file, departing_place] = 1

# Concatenate the aircraft and departing place one-hot encodings
combined_one_hot_encoding = pd.concat([one_hot_encoded_df, one_hot_encoded_departing_places], axis=1)

# Display the first few rows of the combined one-hot encoded DataFrame
combined_one_hot_encoding.head()

,VTICN,VTICQ,VTICF,VTBBJ,VTBBT,VTAIO,VTAIB,VTXEI,VTXER,VTXEQ,...,VTBBX,VTICO,VTXEF,HYD,DEL,AUH,MCT,BLR,CCU,BOM
Output_VTICD_DEL_1105.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Output_VTXEQ_DEL_743.csv,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
Output_VTXER_AUH_1607.csv,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
Output_VTICN_HYD_995.csv,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
Output_VTXEI_DEL_945.csv,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [107]:
combined_one_hot_encoding.shape

(210, 49)

In [108]:
len(unique_departing_places)

7

In [109]:
print(combined_one_hot_encoding.loc['Output_VTAIA_BLR_1439.csv'])

VTICN    0
VTICQ    0
VTICF    0
VTBBJ    0
VTBBT    0
VTAIO    0
VTAIB    0
VTXEI    0
VTXER    0
VTXEQ    0
VTBBQ    0
VTAIH    0
VTXED    0
VTXEO    0
VTXEG    0
VTAIA    1
VTBBK    0
VTICG    0
VTXEP    0
VTBBM    0
VTICD    0
VTAII    0
VTAIG    0
VTBBE    0
VTAIF    0
VTXEN    0
VTAIR    0
VTXEX    0
VTXEU    0
VTBBV    0
VTXEM    0
VTXES    0
VTAIP    0
VTICM    0
VTXDX    0
VTXEL    0
VTBBD    0
VTXEA    0
VTXET    0
VTBBX    0
VTICO    0
VTXEF    0
HYD      0
DEL      0
AUH      0
MCT      0
BLR      1
CCU      0
BOM      0
Name: Output_VTAIA_BLR_1439.csv, dtype: int64


##### Now calculating the delay time

In [110]:
# Function to calculate the delay time for each optimal schedule file
def calculate_delay(file_name, original_schedule):
    # Extract the delay time from the file name
    delay_time = int(file_name.split('_')[-1].split('.')[0])
    
    # Extract the tail (aircraft name) and departing place from the file name
    tail, dep = file_name.split('_')[1], file_name.split('_')[2]

    # Filter the original schedule for rows with the same tail and dep
    matching_flights = original_schedule[(original_schedule['TAIL'] == tail) & (original_schedule['DEP'] == dep)]
    
    # Function to extract hour and minute from 'RTDIST' and calculate delay
    def get_delay(rtdist):
        datetime_obj = pd.to_datetime(rtdist)
        time_in_minutes = datetime_obj.hour * 60 + datetime_obj.minute
        return abs(time_in_minutes - delay_time)
    
    # Calculate the delay for each matching flight
    delays = matching_flights['RTDIST'].apply(get_delay)
    
    # Return the average delay for the schedule file
    return delays.mean() if not delays.empty else 0

In [111]:
# Initialize a list to store the delay times for each optimal schedule file
delays_list = []

# Iterate over each file and calculate the delay time
for file in optimal_schedule_files:
    file_path = os.path.join(optimal_schedules_folder_path, file)
    delay_time = calculate_delay(file, original_schedule)
    delays_list.append(delay_time)

# Check if the length of the list is 210 as expected
len_delays_list = len(delays_list)

len_delays_list, delays_list[:10]  # Display the length and the first 10 elements of the list

(210, [377.5, 327.5, 1532.0, 635.0, 252.5, 299.0, 305.0, 729.0, 97.0, 1093.0])

In [112]:
optimal_schedules_folder_path = '/kaggle/input/optimal-nov-2'
optimal_schedule_files = [file for file in os.listdir(optimal_schedules_folder_path) if file.endswith('.csv')]

In [113]:
len(optimal_schedule_files), optimal_schedule_files[:5]

(210,
 ['Output_VTICD_DEL_1105.csv',
  'Output_VTXEQ_DEL_743.csv',
  'Output_VTXER_AUH_1607.csv',
  'Output_VTICN_HYD_995.csv',
  'Output_VTXEI_DEL_945.csv'])

In [114]:
# Create a list to store file names and their corresponding delay times
delays_data = []

# Populate the list with file names and calculated delay times
for file in optimal_schedule_files:
    delay_time = calculate_delay(file, original_schedule)
    delays_data.append({"File Name": file, "Delay Time": delay_time})

# Convert the list of dictionaries into a DataFrame
optimal_schedule_delays_df = pd.DataFrame(delays_data)

# Now optimal_schedule_delays_df contains 210 rows, each corresponding to an optimal schedule file
# with its calculated delay time


In [115]:
optimal_schedule_delays_df

,File Name,Delay Time
0,Output_VTICD_DEL_1105.csv,377.5
1,Output_VTXEQ_DEL_743.csv,327.5
2,Output_VTXER_AUH_1607.csv,1532.0
3,Output_VTICN_HYD_995.csv,635.0
4,Output_VTXEI_DEL_945.csv,252.5
...,...,...
205,Output_VTBBM_BOM_1138.csv,528.0
206,Output_VTXEU_BOM_846.csv,111.0
207,Output_VTAIR_DEL_675.csv,287.5
208,Output_VTBBM_BOM_1349.csv,739.0


In [116]:
optimal_schedule_delays_df.loc[optimal_schedule_delays_df['File Name'] == 'Output_VTXEU_BOM_1357.csv', 'Delay Time']

53    622.0
Name: Delay Time, dtype: float64

In [117]:
optimal_schedule_files[51]

'Output_VTAIA_BLR_1280.csv'

In [118]:
combined_one_hot_encoding.shape

(210, 49)

In [119]:
optimal_schedule_delays_df

,File Name,Delay Time
0,Output_VTICD_DEL_1105.csv,377.5
1,Output_VTXEQ_DEL_743.csv,327.5
2,Output_VTXER_AUH_1607.csv,1532.0
3,Output_VTICN_HYD_995.csv,635.0
4,Output_VTXEI_DEL_945.csv,252.5
...,...,...
205,Output_VTBBM_BOM_1138.csv,528.0
206,Output_VTXEU_BOM_846.csv,111.0
207,Output_VTAIR_DEL_675.csv,287.5
208,Output_VTBBM_BOM_1349.csv,739.0


In [120]:
optimal_schedule_delays_df['Delay Time'].isna().sum()

0

In [121]:
combined_one_hot_encoding['Delay Time'] = optimal_schedule_delays_df['Delay Time'].values

In [122]:
combined_one_hot_encoding

,VTICN,VTICQ,VTICF,VTBBJ,VTBBT,VTAIO,VTAIB,VTXEI,VTXER,VTXEQ,...,VTICO,VTXEF,HYD,DEL,AUH,MCT,BLR,CCU,BOM,Delay Time
Output_VTICD_DEL_1105.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,377.5
Output_VTXEQ_DEL_743.csv,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,327.5
Output_VTXER_AUH_1607.csv,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1532.0
Output_VTICN_HYD_995.csv,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,635.0
Output_VTXEI_DEL_945.csv,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,252.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Output_VTBBM_BOM_1138.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,528.0
Output_VTXEU_BOM_846.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,111.0
Output_VTAIR_DEL_675.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,287.5
Output_VTBBM_BOM_1349.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,739.0


In [123]:
combined_one_hot_encoding.columns

Index(['VTICN', 'VTICQ', 'VTICF', 'VTBBJ', 'VTBBT', 'VTAIO', 'VTAIB', 'VTXEI',
       'VTXER', 'VTXEQ', 'VTBBQ', 'VTAIH', 'VTXED', 'VTXEO', 'VTXEG', 'VTAIA',
       'VTBBK', 'VTICG', 'VTXEP', 'VTBBM', 'VTICD', 'VTAII', 'VTAIG', 'VTBBE',
       'VTAIF', 'VTXEN', 'VTAIR', 'VTXEX', 'VTXEU', 'VTBBV', 'VTXEM', 'VTXES',
       'VTAIP', 'VTICM', 'VTXDX', 'VTXEL', 'VTBBD', 'VTXEA', 'VTXET', 'VTBBX',
       'VTICO', 'VTXEF', 'HYD', 'DEL', 'AUH', 'MCT', 'BLR', 'CCU', 'BOM',
       'Delay Time'],
      dtype='object')

In [132]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

X = combined_one_hot_encoding
y = Y[1]  # Target column

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier()

# Train the model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate and print the precision and recall
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print(f"Precision: {precision}")
print(f"Recall: {recall}")


Accuracy: 1.0
Precision: 1.0
Recall: 1.0


In [131]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Create a Logistic Regression Classifier
logistic_classifier = LogisticRegression(max_iter=1000, random_state=42)

# Train the Logistic Regression model
logistic_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_logistic = logistic_classifier.predict(X_test)

# Calculate and print the accuracy for Logistic Regression
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
print(f"Logistic Regression Accuracy: {accuracy_logistic}")# Calculate and print the precision and recall for Logistic Regression
precision_logistic = precision_score(y_test, y_pred_logistic)
recall_logistic = recall_score(y_test, y_pred_logistic)
print(f"Logistic Regression Precision: {precision_logistic}")
print(f"Logistic Regression Recall: {recall_logistic}")

Logistic Regression Accuracy: 1.0
Logistic Regression Precision: 1.0
Logistic Regression Recall: 1.0


In [130]:
from sklearn.neural_network import MLPClassifier

# Create an MLP (Multi-Layer Perceptron) Classifier with an additional hidden layer
mlp_classifier = MLPClassifier(hidden_layer_sizes=(128,64), max_iter=1000, random_state=42)

# Train the MLP model
mlp_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_mlp = mlp_classifier.predict(X_test)

# Calculate and print the accuracy for the MLP Classifier
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print(f"MLP Classifier Accuracy : {accuracy_mlp}")

MLP Classifier Accuracy : 0.9047619047619048


In [ ]:
len(y_pred_mlp)

In [ ]:
Y

In [ ]:
Y.shape[1]

In [ ]:
# Create a list to store the trained models and accuracies
mlp_models = []
accuracies = []

# Loop through each column in Y and train a model
for i in range(Y.shape[1]):
    # Extract the target column for training
    y = Y[i]
    
    # Split the data into training (80%) and testing (20%) sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create an MLPClassifier for this column
    mlp_classifier = MLPClassifier(hidden_layer_sizes=(128,64), max_iter=1000, random_state=42)
    
    # Train the model
    mlp_classifier.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = mlp_classifier.predict(X_test)
    
    # Calculate and print the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"MLPClassifier_{i + 1} Accuracy: {accuracy}")
    
    # Append the trained model to the list
    mlp_models.append(mlp_classifier)
    
    # Append the accuracy to the accuracies list
    accuracies.append(accuracy)